In [1]:
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode
from PIL import Image

from torch.utils.data import DataLoader

import torch
import torchmetrics
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import numpy as np

from dataset.cub import CUB200
from util import WarmupLinearSchedule
from pl_model.xfg import LitXFGCrossAttn

In [3]:
train_transform=transforms.Compose([
    transforms.Resize((600, 600), InterpolationMode.BILINEAR),
    transforms.RandomCrop((448, 448)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
train_set = CUB200(root="./data", train=True, caption=True, transform=train_transform)

In [4]:
# model = XFG(config)
# model.load_from(np.load(config.pretrained_dir))
# imgs, txts, targets = train_set[0]
# imgs = torch.Tensor(imgs).unsqueeze(0)
# txts = torch.Tensor(txts)
# model(imgs, txts).shape

In [5]:
if config.logger:
    from pytorch_lightning.loggers import WandbLogger
    logger = WandbLogger(
        project="xfg",
        name=f"vit"
    )
else:
    logger = pl.loggers.TestTubeLogger(
        "output", name=f"vit")
    logger.log_hyperparams(config)

pl.seed_everything(config.seed)
trainer = pl.Trainer(
    precision=16,
    deterministic=True,
    check_val_every_n_epoch=1,
    gpus=config.gpus,
    logger=logger,
    max_epochs=config.epoch,
    weights_summary="top",
    # accelerator='ddp',
)

model = LitXFGCrossAttn(config)
trainer.fit(model)
trainer.test()

Global seed set to 42
/home/lorenzo/anaconda3/envs/xfg/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: You requested multiple GPUs but did not specify a backend, e.g. `Trainer(accelerator="dp"|"ddp"|"ddp2")`. Setting `accelerator="ddp_spawn"` for you.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.
load_pretrained: grid-size from 7 to 18
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: lorenzopark (use `wandb login --relogin` to force relogin)


KeyboardInterrupt: 